# Development Problems 1

**Author**: Omar David Toledo Leguizamón

In [1]:
import numpy as np

Let $D = \{d1, . . . , dn\}$ be a set of documents and $T = \{t1, . . . , tm\}$ a set of terms (words). Let $TD = (T D_{i,j} )_{i=1...m,j=1...n}$ be a matrix such that $T D_{i,j}$ corresponds to the number of times the term $t_i$ appears in the document $d_j$. Also, let li be the length, number of characters, of term $t_i$, and let $L = (l_1, ... , l_m)$ be a column vector. Finally, assume a process where a document $d_j$ is randomly chosen with uniform probability and then a term $t_i$, present in $d_j$ ,is randomly chosen with a probability proportional to the frequency of $t_i$ in $d_j$

## Formal mathematical expression

In order to express the development of the problems using linear algebra operations, we are going to define some constants and functions that may help us to simplify further specification.

### Auxiliary expressions

**1. Define a matrix full of ones**: Define a matrix of size $m \times n$ full of ones as the following:

$$\textbf{1}_{m \times n} = \begin{bmatrix}
1 & \dots  & 1\\
\vdots & \ddots & \vdots \\
1 & \dots & 1
\end{bmatrix}_{m \times n}
$$

**2. Sum by rows**: We define a function that given a matrix of size $m \times n$, we can get a column vector $m \times 1$ that has the sum of the values on each row:


\begin{array}{crl}
      \text{SR: } & \mathbb{R}^{m \times n}&\longrightarrow & \mathbb{R}^{m \times 1} \\
      & A_{m \times n} & \longmapsto & A_{m \times n} \cdot \textbf{1}_{n \times 1} \\
\end{array}


**2. Sum by columns**: We define a function that given a matrix of size $m \times n$, we can get a row vector $1 \times n$ that has the sum of the values on each column:


\begin{array}{crl}
      \text{SC: } & \mathbb{R}^{m \times n}&\longrightarrow & \mathbb{R}^{1 \times n} \\
      & A_{m \times n} & \longmapsto & \textbf{1}_{1 \times m} \cdot A_{m \times n}\\
\end{array}

**3. Convert vector to diagonal matrix**: Given a row vector of size $1 \times n$, we define a function that returns the diagonal matrix of size $n \times n$ whose diagonal is the input vector. This process is done by applying the Hadamard product over a identity matrix

\begin{array}{crl}
      \text{Dg: } & \mathbb{R}^{1 \times n}&\longrightarrow & \mathbb{R}^{n \times n} \\
      & A_{1 \times n} & \longmapsto & \left( \textbf{1}_{n \times 1} \cdot A_{1 \times n} \right) \circ I_n\\
\end{array}

In [2]:
def SR(A):
  return A @ np.ones(shape = (A.shape[1],1))

def SC(A):
  return np.ones(shape = (1,A.shape[0])) @ A

def Dg(A):
  _ , n = A.shape
  return (np.ones(shape = (n,1)) @ A) * np.eye(n)

### Problems to solve

#### $\text{a.  } P(D)$

As it is said in the problem, choosing any document has the same probability. Then if we have $n$ documents, each document has a probability of $1/n$ to be chosen. Then:

$$ P(D)_{1 \times n} =  \dfrac{1}{n} \cdot \textbf{1}_{1 \times n}$$

#### $\text{b.  } P(T|D)$

As it is said in the problem, the probability to choose a term is proportional to the frecuence of the term in the document. Then, the idea will be to get the total amount of terms in each document and then divide the corresponding column defined by the document by this number. As a linear algebra operation, it can be done like:

$$P(T | D)_{m \times n} = {TD}_{m \times n} \cdot \left(\text{Dg}(\text{SC}(TD)) \right)^{-1} _{n \times n}$$

#### $\text{c.   } P(T,D)$

To define the joint probability, we will use the definition given by the product rule. Which says that:

$$P(X,Y) = P(X|Y)P(Y) = P(Y|X)P(X)$$

Then as we already know what is $P(D)$ and $P(T|D)$, we will find the joint probability matrix by applying:

$$P(T,D)_{m \times n} = P(T,D)_{m \times n} \cdot \text{Dg}(P(D))_{n \times n}$$

#### $\text{d.   } P(T)$



In order to get the mmarginal probability of the terms, we are going to use the sum rule. Which says that:

$$P(X) = \sum_{y} P(x,y)$$

As we already have the matrix that represents the joint probability, we can get the marginal probability for the terms by doing:

$$P(T)_{m \times 1} = \text{SR}(P(T,D))_{m \times 1} $$

#### $\text{e.   }P(D | T)$

To get the matrix related to this conditional probability, we are going to use Bayes theorem; which says:

$$P(X|Y) = \dfrac{P(Y|X)P(X)}{P(Y)} = \dfrac{P(X,Y)}{P(Y)}$$

For our purposes, instead of dividing (As there is not a formal expression for this operation in linear algebra) we are going to multiply by the inverse matrix of the marginal probability.

$$P(D|T)_{m \times n} = \left( \text{Dg}(P(T)^{T})\right)^{-1}_{m \times m} \cdot P(T,D)_{m \times n} $$



#### $\text{f.   }E[l]$

In order to get an expected value in a discrete probability space, we must use the following formula:

$$E[f(x)] = \sum_{x} P(x)f(x)$$

For our problem, as we want to get the expected value for the length of the terms, we are going to use the marginal probability related to the terms. Then, the expected value operation can be modeled as a simple dot product. Then:

$$E[l]_{1 \times 1} = l_{1 \times m} \cdot P(T)_{m \times 1}$$

#### $\text{g.   Var}[l]$

The expression that let us get the variance is the following:

$$\text{Var}[f(x)] = E[(f(x) - E[f(x)])^2] = E[f(x)^2] - (E[f(x)])^2$$

In this order of ideas, we express the squared value of $l$ using the Hadamard product (element by element squared).

Then, we can apply the following:

$$\text{Var}[l]_{1 \times 1}=  (l \circ l)_{1 \times m} \cdot P(T)_{m \times 1} - (E[l])^2$$

## Implementation

In [3]:
TD = [[2,3,0,3,7],
      [0,5,5,0,3],
      [5,0,7,3,3],
      [3,1,0,9,9],
      [0,0,7,1,3],
      [6,9,4,6,0]]

L = [5,2,3,6,4,3]

TD = np.array(TD)
L = np.array(L).reshape((1,len(L)))

print(f'TD shape: {TD.shape}')
print(f'L shape: {L.shape}')

TD shape: (6, 5)
L shape: (1, 6)


### Using full linear algebra

#### $\text{a.  } P(D)$

In [4]:
def P_D(TD):
  m,n = TD.shape
  return 1/n * np.ones(shape = (1,n))

P_D(TD)

array([[0.2, 0.2, 0.2, 0.2, 0.2]])

#### $\text{b.  } P(T|D)$

In [5]:
def P_T_given_D(TD):
  return TD @ np.linalg.inv(Dg(SC(TD)))

P_T_given_D(TD)

array([[0.125     , 0.16666667, 0.        , 0.13636364, 0.28      ],
       [0.        , 0.27777778, 0.2173913 , 0.        , 0.12      ],
       [0.3125    , 0.        , 0.30434783, 0.13636364, 0.12      ],
       [0.1875    , 0.05555556, 0.        , 0.40909091, 0.36      ],
       [0.        , 0.        , 0.30434783, 0.04545455, 0.12      ],
       [0.375     , 0.5       , 0.17391304, 0.27272727, 0.        ]])

#### $\text{c.  } P(T,D)$

In [6]:
def P_T_D_jopint(TD):
  return P_T_given_D(TD) @ Dg(P_D(TD))

P_T_D_jopint(TD)

array([[0.025     , 0.03333333, 0.        , 0.02727273, 0.056     ],
       [0.        , 0.05555556, 0.04347826, 0.        , 0.024     ],
       [0.0625    , 0.        , 0.06086957, 0.02727273, 0.024     ],
       [0.0375    , 0.01111111, 0.        , 0.08181818, 0.072     ],
       [0.        , 0.        , 0.06086957, 0.00909091, 0.024     ],
       [0.075     , 0.1       , 0.03478261, 0.05454545, 0.        ]])

#### $\text{d.  } P(T)$

In [7]:
def P_T(TD):
  return SR(P_T_D_jopint(TD))

P_T(TD)

array([[0.14160606],
       [0.12303382],
       [0.17464229],
       [0.20242929],
       [0.09396047],
       [0.26432806]])

#### $\text{e.  } P(D|T)$

In [8]:
def P_D_given_T(TD):
  return np.linalg.inv(Dg(P_T(TD).T)) @ P_T_D_jopint(TD)

P_D_given_T(TD)

array([[0.17654612, 0.23539482, 0.        , 0.19259576, 0.3954633 ],
       [0.        , 0.45154704, 0.35338464, 0.        , 0.19506832],
       [0.35787437, 0.        , 0.34853851, 0.15616336, 0.13742376],
       [0.18524987, 0.05488885, 0.        , 0.40418153, 0.35567975],
       [0.        , 0.        , 0.64782097, 0.09675248, 0.25542655],
       [0.28373832, 0.37831776, 0.13158879, 0.20635514, 0.        ]])

#### $\text{f.  } E[l]$

In [9]:
def expected_length(TD,L):
  return L @ P_T(TD)

expected_length(TD,L)

array([[3.86142666]])

#### $\text{f.  Var}[l]$

In [10]:
def variance_length(TD,L):
  return (L)**2 @ P_T(TD) - (expected_length(TD,L))**2

variance_length(TD,L)

array([[1.86322628]])

### Using some implemented operations in numpy

#### $\text{a.  } P(D)$

In [11]:
def P_D(TD):
  m,n = TD.shape
  return np.full(shape = (1,n), fill_value = 1/n)

P_D(TD)

array([[0.2, 0.2, 0.2, 0.2, 0.2]])

#### $\text{b.  } P(T|D)$

In [12]:
def P_T_given_D(TD):
  sums = TD.sum(axis=0)
  return TD / sums

P_T_given_D(TD)

array([[0.125     , 0.16666667, 0.        , 0.13636364, 0.28      ],
       [0.        , 0.27777778, 0.2173913 , 0.        , 0.12      ],
       [0.3125    , 0.        , 0.30434783, 0.13636364, 0.12      ],
       [0.1875    , 0.05555556, 0.        , 0.40909091, 0.36      ],
       [0.        , 0.        , 0.30434783, 0.04545455, 0.12      ],
       [0.375     , 0.5       , 0.17391304, 0.27272727, 0.        ]])

#### $\text{c.  } P(T,D)$

In [13]:
def P_T_D_joint(TD):
  return P_T_given_D(TD) * P_D(TD)

P_T_D_joint(TD)

array([[0.025     , 0.03333333, 0.        , 0.02727273, 0.056     ],
       [0.        , 0.05555556, 0.04347826, 0.        , 0.024     ],
       [0.0625    , 0.        , 0.06086957, 0.02727273, 0.024     ],
       [0.0375    , 0.01111111, 0.        , 0.08181818, 0.072     ],
       [0.        , 0.        , 0.06086957, 0.00909091, 0.024     ],
       [0.075     , 0.1       , 0.03478261, 0.05454545, 0.        ]])

#### $\text{d.  } P(T)$

In [14]:
def P_T(TD):
  return P_T_D_jopint(TD).sum(axis=1)

P_T(TD)

array([0.14160606, 0.12303382, 0.17464229, 0.20242929, 0.09396047,
       0.26432806])

#### $\text{e.  } P(D|T)$

In [15]:
def P_D_given_T(TD):
  return  (P_T_D_joint(TD).T / P_T(TD)).T

P_D_given_T(TD)

array([[0.17654612, 0.23539482, 0.        , 0.19259576, 0.3954633 ],
       [0.        , 0.45154704, 0.35338464, 0.        , 0.19506832],
       [0.35787437, 0.        , 0.34853851, 0.15616336, 0.13742376],
       [0.18524987, 0.05488885, 0.        , 0.40418153, 0.35567975],
       [0.        , 0.        , 0.64782097, 0.09675248, 0.25542655],
       [0.28373832, 0.37831776, 0.13158879, 0.20635514, 0.        ]])

#### $\text{f.  } E[l]$

In [16]:
def expected_length(TD,L):
  return L @ P_T(TD)

expected_length(TD,L)

array([3.86142666])

#### $\text{f.  Var}[l]$

In [17]:
def variance_length(TD,L):
  return (L)**2 @ P_T(TD) - (expected_length(TD,L))**2

variance_length(TD,L)

array([1.86322628])